# Algoritmo

En este fichero se almacena el algoritmo grasp y las funciones necesarias para su ejecución.

---
## Librerías
- numpy: necesaria para trabajar de forma eficiente con arrays.
- random: necesaria para la generación de números aleatorios.
- math: necesaria para las funciones matemáticas.
- itertools: necesaria para funciones matemáticas.
- sys: necesaria para la generación del máximo entero que se puede almacenar.
- time: necesaria para la generación de estadisticas, en concreto, para visualizar el tiempo de ejecución del algoritmo.

In [1]:
import numpy as np
import random
import math
import itertools
import sys
import time

---
## Función para calcular la matriz de costes

In [2]:
def calcular_matriz_de_costes(dataset):
    n_ciudades = len(dataset)
    distancias = np.zeros((n_ciudades,n_ciudades))
    for i in range(n_ciudades):
        for j in range(n_ciudades):
            if(i != j):
                distancias[i][j] = calcular_distancia_euclidea(dataset[i],dataset[j])
    return distancias

In [3]:
def calcular_distancia_euclidea(ciudad_1,ciudad_2):
    
    x1,y1 = ciudad_1[1 :]
    x2,y2 = ciudad_2[1 :]
    
    xd = x1 - x2
    yd = y1 - y2
    
    return round(math.sqrt( xd*xd + yd*yd ))

---
## Busqueda local: el primer mejor vecino

## Función para calcular el coste de un camíno en función de la matriz de costes

In [4]:
def calcular_coste(camino,distancias):
    coste = 0
    for indice in range(len(camino)-1):
        coste += distancias[camino[indice]][camino[indice+1]]
    coste += distancias[camino[-1]][camino[0]]
    return coste

In [5]:
def generar_array_ciudades_cercanas_a_vecinos(combinaciones_indices_vecinos,len_dataset):
    ciudades_cercanas = []

    for combinacion in combinaciones_indices_vecinos:
        if (combinacion[0] == 0):
            a1 = len_dataset - 1
        else:
            a1 = combinacion[0] - 1

        if (combinacion[0] == len_dataset - 1):
            s1 = 0
        else:
            s1 = combinacion[0] + 1

        if (combinacion[1] == 0):
            a2 = len_dataset - 1
        else:
            a2 = combinacion[1] - 1

        if (combinacion[1] == len_dataset - 1):
            s2 = 0
        else:
            s2 = combinacion[1] + 1

        ciudades_cercanas.append([a1, s1, a2, s2])

    return ciudades_cercanas

In [6]:
def calcular_coste_intercambio(mejor_coste,mejor_solucion,vecinos,intercambio,len_dataset,matriz_de_costes):
    # Por cada vecino, estudiaremos su coste
    coste_candidato = mejor_coste

    # Primero, por comodidad, generaremos los indices del camino
    a1 = vecinos[0]
    s1 = vecinos[1]
    a2 = vecinos[2]
    s2 = vecinos[3]
    i1 = intercambio[0]
    i2 = intercambio[1]

    if (i2 - 1 == i1):

        # A continuación eliminaremos el coste generado por las conexiones de nodos a intercambiar

        coste_candidato -= matriz_de_costes[mejor_solucion[a1]][mejor_solucion[i1]]
        coste_candidato -= matriz_de_costes[mejor_solucion[i1]][mejor_solucion[i2]]
        coste_candidato -= matriz_de_costes[mejor_solucion[i2]][mejor_solucion[s2]]

        # Despues sumamos el coste del nuevo camino

        coste_candidato += matriz_de_costes[mejor_solucion[a1]][mejor_solucion[i2]]
        coste_candidato += matriz_de_costes[mejor_solucion[i2]][mejor_solucion[i1]]
        coste_candidato += matriz_de_costes[mejor_solucion[i1]][mejor_solucion[s2]]

    elif(i1 == 0 and i2 == len_dataset-1):
        # A continuación eliminaremos el coste generado por las conexiones de nodos a intercambiar

        coste_candidato -= matriz_de_costes[mejor_solucion[a2]][mejor_solucion[i2]]
        coste_candidato -= matriz_de_costes[mejor_solucion[i2]][mejor_solucion[i1]]
        coste_candidato -= matriz_de_costes[mejor_solucion[i1]][mejor_solucion[s1]]

        # Despues sumamos el coste del nuevo camino

        coste_candidato += matriz_de_costes[mejor_solucion[a2]][mejor_solucion[i1]]
        coste_candidato += matriz_de_costes[mejor_solucion[i1]][mejor_solucion[i2]]
        coste_candidato += matriz_de_costes[mejor_solucion[i2]][mejor_solucion[s1]]
    else:

        # A continuación eliminaremos el coste generado por las conexiones de nodos a intercambiar

        coste_candidato -= matriz_de_costes[mejor_solucion[a1]][mejor_solucion[i1]]
        coste_candidato -= matriz_de_costes[mejor_solucion[i1]][mejor_solucion[s1]]

        coste_candidato -= matriz_de_costes[mejor_solucion[a2]][mejor_solucion[i2]]
        coste_candidato -= matriz_de_costes[mejor_solucion[i2]][mejor_solucion[s2]]

        # Despues sumamos el coste del nuevo camino

        coste_candidato += matriz_de_costes[mejor_solucion[a1]][mejor_solucion[i2]]
        coste_candidato += matriz_de_costes[mejor_solucion[i2]][mejor_solucion[s1]]

        coste_candidato += matriz_de_costes[mejor_solucion[a2]][mejor_solucion[i1]]
        coste_candidato += matriz_de_costes[mejor_solucion[i1]][mejor_solucion[s2]]
    
    return coste_candidato

In [7]:
def busqueda_local_el_mejor_vecino(solucion,matriz_de_costes,len_dataset):
    
    mejor_solucion = solucion                                           # Generación de solución inicial
    mejor_coste = calcular_coste(mejor_solucion, matriz_de_costes)      # Generación del coste inicial

    # Número máximo de evaluaciones, segunda condición de parada del bucle.
    num_evaluacion = len_dataset*1600

    # Indices de todos los posibles intercambios de dos ciudades del dataset [ (0,1), (0,2), ... , (10,23), ...  ]
    intercambios_de_dos_ciudades = list(itertools.combinations(list(range(0, len_dataset)), 2))
    len_intercambios_de_dos_ciudades = len(intercambios_de_dos_ciudades)

    # Indices de los vecinos de los indices del array intercambios_de_dos_ciudades : 
    # intercambios_de_dos_ciudades[x] = (6,10)
    # vecinos_de_intercambios_de_dos_ciudades = [5,7,9,11]
    vecinos_de_intercambios_de_dos_ciudades = generar_array_ciudades_cercanas_a_vecinos(intercambios_de_dos_ciudades,len_dataset)

    
    seguir = True       # Variable de salida del bucle
    evaluacion = 0      # Nº de evaluaciones actual del bucle

    while (seguir):

        intercambio_actual = ()
        coste_actual = mejor_coste

        for i in range(len_intercambios_de_dos_ciudades):

            coste_candidato = calcular_coste_intercambio(mejor_coste,mejor_solucion,vecinos_de_intercambios_de_dos_ciudades[i],intercambios_de_dos_ciudades[i],len_dataset,matriz_de_costes)

            evaluacion += 1
            
            if (coste_candidato < coste_actual):
                coste_actual = coste_candidato
                intercambio_actual = intercambios_de_dos_ciudades[i]

            if evaluacion >= num_evaluacion:
                break
                
        if intercambio_actual == ():
            seguir = False
        else:
            buffer = mejor_solucion[intercambio_actual[0]]
            mejor_solucion[intercambio_actual[0]] = mejor_solucion[intercambio_actual[1]]
            mejor_solucion[intercambio_actual[1]] = buffer
            mejor_coste = coste_actual

        if evaluacion >= num_evaluacion:
            seguir = False

    return mejor_solucion, mejor_coste, evaluacion

---
## Greedy aleatorizado

In [8]:
def greedy_aleatorizado(matriz_de_costes,num_indices):

    indice = random.randint(0,num_indices-1)

    ciudades_no_analizadas = list(range(num_indices))
    solucion_actual = [ciudades_no_analizadas[indice]]
    ciudades_no_analizadas.pop(indice)

    while(len(ciudades_no_analizadas) > 0):
        
        l = math.ceil(0.1 * len(ciudades_no_analizadas))

        costes = []
        for ciudad in ciudades_no_analizadas:
            costes.append( matriz_de_costes[solucion_actual[-1]][ciudad])
        
        menores_indices = list(np.argsort(costes))[:l] # Indices del array ordenados por coste
        costes_candidatos = [costes[indice] for indice in menores_indices] # Costes para cada ciudad
        
        costes_invertidos = [ 1/coste for coste in costes_candidatos]
        total = sum(costes_invertidos)
        
        probabilidades = [coste/total for coste in costes_invertidos]
        aleatorio = random.uniform(0,1)

        encontrado = False
        indice = 0

        while(not encontrado and indice == len(probabilidades)):
            if(aleatorio <= probabilidades[indice]):
                encontrado = True
            else:
                aleatorio-=probabilidades[indice]
                indice+=1

        if(indice == len(probabilidades)):
            indice -=1
        
        solucion_actual.append(ciudades_no_analizadas[menores_indices[indice]])
        ciudades_no_analizadas.remove(ciudades_no_analizadas[menores_indices[indice]])
        

    return solucion_actual

---
## GRASP

In [9]:
def grasp(num_ejecuciones,semilla,dataset):
    estadisticas = []
    matriz_de_costes = calcular_matriz_de_costes(dataset)
    mejor_solucion = []
    mejor_coste = sys.maxsize

    random.seed = semilla
    
    len_dataset = len(dataset)

    for indice in range(num_ejecuciones):
        
        inicio = time.time()
        
        solucion_candidata = greedy_aleatorizado(matriz_de_costes,len_dataset)
        coste_candidato = calcular_coste(solucion_candidata,matriz_de_costes)
        
        solucion_candidata_optimizada, coste_candidato_optimizado, evaluaciones = busqueda_local_el_mejor_vecino(solucion_candidata,matriz_de_costes,len_dataset)

        if(coste_candidato_optimizado < mejor_coste):
            mejor_solucion = solucion_candidata_optimizada
            mejor_coste = coste_candidato_optimizado
    
        fin = time.time()
        
        estadisticas.append([coste_candidato,coste_candidato_optimizado,mejor_coste,evaluaciones + 1, fin-inicio])
    
    return [dataset[ciudad] for ciudad in mejor_solucion], mejor_coste, estadisticas